In [43]:
from PIL import Image
import numpy as np
import pandas as pd
import pandas_datareader as pdr
import os

import time
import progressbar
import talib

In [70]:
# change height

def img_graph(input_data, height = 64):
    
    data = input_data.copy()
    
    lenght = 3 * len(data)
    
    im = Image.new(mode = '1', size = (lenght, height))
    
    volume_height = round(height * 0.2)
    volume_max =  data['VOL'].max()
    vol = round((data['VOL'] / volume_max) * volume_height - volume_height * 0.05)
    
    vol[vol == 0] = 1
    vol[vol < 0] = 1
    vol[vol == vol.max()] = vol.max() - 1
    data.loc[:, 'VOL'] = vol
    
    maximum = data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']].max().max()
    minimum = data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']].min().min()
    
    price_height = height# - volume_height
    data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']] = round(((data.loc[:, ['OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'MA']] - minimum)/(maximum - minimum)) * price_height)
    
    matrix = pd.DataFrame(np.array(im))

    for i,j in zip(range(1, lenght - 1, 3), data.index):
        
        try:
            o = price_height - int(data.loc[j,'OPENPRC'])
            h = price_height - int(data.loc[j,'ASKHI'])
            l = price_height - int(data.loc[j,'BIDLO'])
            c = price_height - int(data.loc[j,'PRC'])
            ma = price_height - int(data.loc[j, 'MA'])
#             ma1 = price_height - int(data.loc[j, 'UP'])
#             ma2 = price_height - int(data.loc[j, 'LOW'])
#             ma3 = price_height - int(data.loc[j, 'MID'])
#             v = -int(data.loc[j, 'VOL'])
            if o < 0 or h < 0 or l < 0 or c < 0:            
                matrix.iloc[v:, int(i)] = True
            else:
                matrix.iloc[int(o),int(i) - 1] = True
                matrix.iloc[int(h):int(l) + 1, int(i)] = True
                matrix.iloc[int(c), int(i) + 1] = True
                matrix.iloc[int(ma), int(i) - 1 : int(i) + 1] = True
#                 matrix.iloc[int(ma1), int(i) - 1 : int(i) + 1] = True
#                 matrix.iloc[int(ma2), int(i) - 1 : int(i) + 1] = True
#                 matrix.iloc[int(ma3), int(i) - 1 : int(i) + 1] = True
#                 matrix.iloc[v:, int(i)] = True
        except: pass
            
# #             matrix.iloc[int(h):int(l) + 1, int(i)] = True
# #             matrix.iloc[int(c), int(i) + 1] = True
# #             matrix.iloc[v:, int(i)] = True
# #         elif h < 0 or l < 0:
# #             matrix.iloc[int(o),int(i) - 1] = True
# #             matrix.iloc[int(c), int(i) + 1] = True
# #             matrix.iloc[v:, int(i)] = True
# #         elif c < 0:
# #             matrix.iloc[int(o),int(i) - 1] = True
# #             matrix.iloc[int(h):int(l) + 1, int(i)] = True
# #             matrix.iloc[v:, int(i)] = True
#         else:
#         matrix.iloc[int(o),int(i) - 1] = True
#         matrix.iloc[int(h):int(l) + 1, int(i)] = True
#         matrix.iloc[int(c), int(i) + 1] = True
#         matrix.iloc[v:, int(i)] = True

    graph = Image.fromarray(np.array(matrix))
#     graph.show()
    return(graph)


In [93]:
meta = pd.read_excel(r"C:\Users\hayk.darbinyan\codes\my_Venvs\_venv-310-jupyter\Thesis\Other__assets\Commodities\commodities.xlsx", index_col = 0, engine = 'openpyxl')
print(len(meta))
meta.dropna(axis = 0, inplace = True)
print(len(meta))
meta = meta.iloc[::-1]
meta.head()

19791
19034


,date,PRC,OPENPRC,BIDLO,ASKHI,VOL
Permno,,,,,,
SOYBEAN,2010-01-04,1049.50,1043.00,1041.25,1065.50,25947.0
SOYBEAN,2010-01-05,1052.25,1047.00,1042.00,1056.00,21073.0
SOYBEAN,2010-01-06,1050.50,1050.00,1042.75,1058.50,17567.0
SOYBEAN,2010-01-07,1017.75,1050.50,1016.50,1052.00,11750.0
SOYBEAN,2010-01-08,1013.00,1018.25,1005.00,1018.25,7820.0


In [96]:
observation_period = 20
height = 64
test_period = 20
subset = 'Test_2016_com'
iters = set(meta.index)
indicator = 'None'
a_count = 0
labels_df = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_20d', 'Signal'])

pbar = progressbar.ProgressBar(maxval = len(iters)).start()

for k, permno in enumerate(list(iters)):
    
    data = meta.loc[permno, :]
#     if data.iloc[0, 0] == '2017-01-03' and data.iloc[-1, 0] == '2022-12-30': # 1994-01-03, 2016-12-30
#         pass
#     else:
#         continue
#     start = '2010-01-03'
#     end = '2016-12-30'
    start = '2017-01-03'
    end = '2022-12-31'
    data = data.loc[:, ['date', 'OPENPRC', 'ASKHI', 'BIDLO', 'PRC', 'VOL']]
    data['MA'] = data['PRC'].rolling(observation_period).mean()
    data['Std'] = data['PRC'].pct_change().rolling(252).std()
#     data['UP'], data['MID'], data['LOW'] = talib.BBANDS(data['PRC'], timeperiod = observation_period)
#     data['VOL'] = talib.MFI(data['ASKHI'], data['BIDLO'], data['PRC'], data['VOL'], timeperiod = observation_period)
    data = data.iloc[observation_period + 1:, :]
#     data['ret_5d'] = np.zeros(len(data))
    data['ret_20d'] = np.zeros(len(data))
#     data['ret_60d'] = np.zeros(len(data))
    data.set_index('date', inplace = True)
    data = data.loc[start:end, :]
    l = len(data)
    iterations = round(l/(test_period)) - 6
    
    pic_path = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path) == True:
        pass
    else:
        os.makedirs(pic_path)

    pic_path1 = f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}"
    
    if os.path.isdir(pic_path1) == True:
        pass
    else:
        os.makedirs(pic_path1)
    
    labels = pd.DataFrame(columns = ['Date', 'Permno', 'Ret_20d', 'Signal'])
    
    for i in range(iterations):
        
        signal = None
        ret = None
        
        start = i * test_period
        end = observation_period + i * test_period
        lookback = data.iloc[start : end, :]
#         lookback_class5 = data.iloc[end : end + 5]
        lookback_class20 = data.iloc[end : end + 20]
        
        openprc20 = lookback_class20.iloc[0, 0]
        closeprc20 = lookback_class20.iloc[-1, 3]
        
        ret20 = (closeprc20 - openprc20)/openprc20

        std = lookback_class20.iloc[0, 6]
        price = lookback_class20.iloc[0, 0]
        
        upper = price * (1 + 1.5 * std)
        lower = price * (1 - 0.5 * std)
        
        hilo = lookback_class20.iloc[:, 1:3]
        
        if hilo['ASKHI'][hilo['ASKHI'] > upper].index.shape[0] == 0:
            up_lim = None
        else:
            up_lim = hilo['ASKHI'][hilo['ASKHI'] > upper].index[0]
        
        if hilo['BIDLO'][hilo['BIDLO'] < lower].index.shape[0] == 0:
            down_lim = None
        else:
            down_lim = hilo['BIDLO'][hilo['BIDLO'] < lower].index[0]
            
        if up_lim != None and down_lim != None:
            if up_lim < down_lim:
                signal = 1
            else:
                signal = 0
        elif up_lim != None and down_lim == None:
            signal = 1
        elif up_lim == None and down_lim != None:
            signal = 0
        else:
            if ret20 > 0:
                signal = 1
            else:
                signal = 0
            
#         lookback_class60 = data.iloc[end : end + 60]

#         print(lookback_class60)
        
#         try:
        
        
        labels.loc[lookback_class20.index[0], :] = [lookback_class20.index[0], permno, ret20, signal]
        
#         except:
#             ret = (int(lookback_class.iloc[-1, :-3].mean()) - int(lookback_class.iloc[0, :-2].mean()))/int(lookback_class.iloc[0, :-2].mean())

        # buy signal -- 1
        # sell signal -- 0
            
        graph = img_graph(lookback, height = height)
        graph.save(f"Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/{subset}/{a_count}.jpeg")
        a_count += 1

    labels.reset_index(inplace = True, drop = True)
    labels_df = pd.concat([labels_df, labels], axis = 0, ignore_index = True)
    pbar.update(k + 1)    
    
pbar.finish()

100% |########################################################################|


In [97]:
labels_df.to_csv(f"C:/Users/hayk.darbinyan/codes/my_Venvs/_venv-310-jupyter/Thesis/Dataset/Pics_{observation_period}_{test_period}_{indicator}_Torch/Labels_{subset}.csv", index = True)